<center><h2>Artificial and Computational Intelligence (Assignment - 2)</h2></center>

## Problem Statement

As part of the 2nd Assignment, we'll implement Bayesian Networks and also learn to use the pomegranate library.

You are required to create a bayesian network model which would help you predict the probability. The detailed problem description is attached as a PDF as a part of this assignment along with the marking scheme.  

### What is a Bayesian Network ?

A Bayesian network, Bayes network, belief network, decision network, Bayes(ian) model or probabilistic directed acyclic graphical model is a probabilistic graphical model (a type of statistical model) that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). 

Bayesian networks are ideal for taking an event that occurred and predicting the likelihood that any one of several possible known causes was the contributing factor. For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. Given symptoms, the network can be used to compute the probabilities of the presence of various diseases. 

### Dataset

The dataset can be downloaded from https://drive.google.com/drive/folders/1oMtKmmvPkN4O8DmrHMJe6M8CbB93Z5kw .You can access it only using your BITS IDs. Also, the same dataset is attached along with the assignment. 

#### Dataset Description
##### Sample Tuple

Y	won	5wickets	lost	2nd	vWest_Indies	Home	6-Nov-11

##### Explanation
- The first column represents if Ashwin was in the playing 11 or not. 
- The second column represents the Result of the match . win indicates India won the match.
- The third column represents the Margin of victory / losss.
- The fourth column represents the results of the toss. won indicates India won the toss. 
- The fifth column represents the batting order. If India batted 1st or 2nd. 
- The sixth column represents the opponent.
- The seventh column represents the location of the match. If the match was held in Home(India) or away. 
- The last column represents the start date of the match.


### Evaluation
We wish to evaluate based on 
- coding practices being followed
- commenting to explain the code and logic behind doing something
- your understanding and explanation of data
- how good the model would perform

Jaikumar T V - 2018AC04556 

Praven H     - 2018AC04536

Sai Vikas    - 2018AC04557

Prasanna R   - 2018AC04552

In [36]:
!pip install -q pomegranate==0.11.2

In [37]:
#Import libraries
import pandas as pd
from pomegranate import *

In [38]:
#Read data
indian_test_stats = pd.read_excel('India_Test_Stats.xlsx')
print(indian_test_stats)

   Ashwin Result           Margin  Toss  Bat      Opposition Location  \
0       Y    won        5 wickets  lost  2nd   v West Indies     Home   
1       Y    won   inns & 15 runs   won  1st   v West Indies     Home   
2       Y   draw                -  lost  2nd   v West Indies     Home   
3       Y   lost         122 runs  lost  2nd     v Australia     Away   
4       Y   lost   inns & 68 runs   won  1st     v Australia     Away   
5       N   lost   inns & 37 runs  lost  1st     v Australia     Away   
6       Y   lost         298 runs  lost  2nd     v Australia     Away   
7       Y    won  inns & 115 runs   won  1st   v New Zealand     Home   
8       Y    won        5 wickets  lost  2nd   v New Zealand     Home   
9       Y    won        9 wickets   won  1st       v England     Home   
10      Y   lost       10 wickets   won  1st       v England     Home   
11      Y   lost        7 wickets   won  1st       v England     Home   
12      Y   draw                -  lost  2nd       

In [39]:
#Pre-process data (Whatever you feel might be required)
# Taking only the required parameters as per the problem statement
indian_test_stats = indian_test_stats[['Ashwin', 'Result', 'Toss', 'Bat', 'Location']]
indian_test_stats.head()
indian_test_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 5 columns):
Ashwin      85 non-null object
Result      85 non-null object
Toss        85 non-null object
Bat         85 non-null object
Location    85 non-null object
dtypes: object(5)
memory usage: 3.4+ KB


In [42]:
#Data Description
indian_test_stats.describe()

,Ashwin,Result,Toss,Bat,Location
count,85,85,85,85,85
unique,2,3,2,2,2
top,Y,won,lost,1st,Home
freq,70,47,45,46,43


###**Construction of Bayesian Network** 
 Independent variables - Location and Toss

 Dependent Variables   - Ashwin playing(Depends on Location), Batting(Depends on Toss) and
                         Result (Depends on Ashwin playing and Batting)
  
 We use the prior probability for Independent variables and conditional probability for dependent variables. The Dependent variables are connected to independent by edges.
 
 Location ---->Ashwin_playing ---> Result

 Toss     ---->Batting        ---> Result
 

In [43]:
#Solution for part 1 

#The function should read in an array and output a dictionary of prior probability of each possible outcome.
def constructAndDisplayPriorProbability (nodes):
    priorProbabilities = {}   
    for node in nodes:
        priorProbabilities[node] = calculatePriorProbability(indian_test_stats[node])
    
    print(" Prior Propability of ashwin playing :")
    print(priorProbabilities['Ashwin'])
    print("\n Prior Propability of Toss :")
    print(priorProbabilities['Toss'])
    print("\n Prior Propability of Result :")
    print(priorProbabilities['Result'])
    print("\n Prior Propability of Bat :")
    print(priorProbabilities['Bat'])
    print("\n Prior Propability of Location :")
    print(priorProbabilities['Location'])
    return priorProbabilities

def calculatePriorProbability (dataObject):
    length = len(dataObject)
    uniqueValues = dataObject.unique()    
    dataList = list(dataObject)    
    prior = {}
    for x in uniqueValues:
        count = dataList.count(x)
        prior[x] = dataList.count(x)/length  
    return prior


nodes = ['Ashwin', 'Result', 'Toss', 'Bat', 'Location']
priorProbabilities = constructAndDisplayPriorProbability(nodes)


 Prior Propability of ashwin playing :
{'Y': 0.8235294117647058, 'N': 0.17647058823529413}

 Prior Propability of Toss :
{'lost': 0.5294117647058824, 'won': 0.47058823529411764}

 Prior Propability of Result :
{'won': 0.5529411764705883, 'draw': 0.18823529411764706, 'lost': 0.25882352941176473}

 Prior Propability of Bat :
{'2nd': 0.4588235294117647, '1st': 0.5411764705882353}

 Prior Propability of Location :
{'Home': 0.5058823529411764, 'Away': 0.49411764705882355}


In [44]:
#Solution for part 2
def calculate_conditional_prob(indian_test_stats, prior_col_name, posterior_col_name ):
    df_s = indian_test_stats.groupby(prior_col_name)[posterior_col_name].value_counts() / indian_test_stats.groupby(prior_col_name)[posterior_col_name].count()
    df_s = df_s.unstack(fill_value=0).stack()
    df_f = df_s.reset_index(name='cpt')    
    return df_f.values.tolist()

def calculate_conditional_prob_for_result(indian_test_stats, prior_col_name1, prior_col_name2, posterior_col_name ):
    df_s = indian_test_stats.groupby([prior_col_name1, prior_col_name2])[posterior_col_name].value_counts() / indian_test_stats.groupby([prior_col_name1, prior_col_name2])[posterior_col_name].count()
    df_s = df_s.unstack(fill_value=0).stack()
    df_f = df_s.reset_index(name='cpt')     
    return df_f.values.tolist()

print(priorProbabilities)
location_prior = DiscreteDistribution(priorProbabilities['Location'])
toss_prior     = DiscreteDistribution(priorProbabilities['Toss'])
ashwin_prior   = DiscreteDistribution(priorProbabilities['Ashwin'])
batting_prior  = DiscreteDistribution(priorProbabilities['Bat'])

ashwin_playing =   ConditionalProbabilityTable(calculate_conditional_prob(indian_test_stats, 'Location', 'Ashwin'), [location_prior]);
print(" Condition Probability of Ashwin playing Node :")
print(ashwin_playing)
batting =          ConditionalProbabilityTable(calculate_conditional_prob(indian_test_stats, 'Toss', 'Bat'), [toss_prior]);
print("\n Condition Probability of Batting Node :")
print(batting)
result =           ConditionalProbabilityTable(calculate_conditional_prob_for_result(indian_test_stats, 'Ashwin', 'Bat', 'Result'), [ashwin_playing, batting]);
print("\n Condition Probability of Result Node :")
print(result)

{'Ashwin': {'Y': 0.8235294117647058, 'N': 0.17647058823529413}, 'Result': {'won': 0.5529411764705883, 'draw': 0.18823529411764706, 'lost': 0.25882352941176473}, 'Toss': {'lost': 0.5294117647058824, 'won': 0.47058823529411764}, 'Bat': {'2nd': 0.4588235294117647, '1st': 0.5411764705882353}, 'Location': {'Home': 0.5058823529411764, 'Away': 0.49411764705882355}}
 Condition Probability of Ashwin playing Node :
Away	N	0.3571428571428572
Away	Y	0.6428571428571429
Home	N	0.0
Home	Y	1.0

 Condition Probability of Batting Node :
lost	1st	0.2222222222222222
lost	2nd	0.7777777777777778
won	1st	0.9
won	2nd	0.10000000000000002

 Condition Probability of Result Node :
N	1st	draw	0.2222222222222222
N	1st	lost	0.2222222222222222
N	1st	won	0.5555555555555556
N	2nd	draw	0.16666666666666669
N	2nd	lost	0.8333333333333334
N	2nd	won	0.0
Y	1st	draw	0.10810810810810814
Y	1st	lost	0.1891891891891892
Y	1st	won	0.7027027027027027
Y	2nd	draw	0.2727272727272727
Y	2nd	lost	0.24242424242424243
Y	2nd	won	0.48484848484

In [45]:
#Solution for part 3 

# state creation
location_state = State(location_prior, name="location_prior")
toss_state     = State(toss_prior, name="toss_prior")
aswin_state    = State(ashwin_playing, name="ashwin_playing")
batting_state  = State(batting, name="batting")
result_state   = State(result, name="result")

# Initialize the model and add all the nodes/states
model = BayesianNetwork("Network to check the probability of Ashwin inclusion in Indian Team")
model.add_states(location_state, toss_state, aswin_state, batting_state, result_state)

# Add the edges
model.add_edge(location_state, aswin_state)
model.add_edge(toss_state, batting_state)
model.add_edge(aswin_state, result_state)
model.add_edge(batting_state, result_state)

#Building the Model
model.bake()

In [46]:
#Solution for part 4
# a) India winning, batting 2nd, Ashwin playing
print (model.predict_proba({'result': 'won', 'batting': '2nd', 'ashwin_playing': 'Y'}))
# Gives the Result probability
print (model.predict_proba({'batting': '2nd', 'ashwin_playing': 'Y'}))


[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.6142857142857142,
            "Away" :0.3857142857142858
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'Y' '2nd' 'won']
[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.6142857142857142,
            "Away" :0.3857142857142858
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'Y' '2nd'
 {
    "class" :"Distributi

In [47]:

# b) India winning, batting 2nd, Ashwin not playing
print (model.predict_proba({'result': 'won', 'batting': '2nd', 'ashwin_playing': 'N'}))
# Gives the Result probability
print (model.predict_proba({'batting': '2nd', 'ashwin_playing': 'N'}))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.5058823529411764,
            "Away" :0.49411764705882366
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.5294117647058824,
            "won" :0.47058823529411764
        }
    ],
    "frozen" :false
}
 'N' '2nd' 'won']
[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.0,
            "Away" :1.0
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'N' '2nd'
 {
    "class" :"Distribution",
    "dtype" :"str",
    

In [48]:
# c) India losing, batting 2nd, Ashwin playing
print (model.predict_proba({'result': 'lost', 'batting': '2nd', 'ashwin_playing': 'Y'}))
# Gives the Result probability
print (model.predict_proba({'batting': '2nd', 'ashwin_playing': 'Y'}))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.6142857142857142,
            "Away" :0.3857142857142858
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'Y' '2nd' 'lost']
[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.6142857142857142,
            "Away" :0.3857142857142858
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'Y' '2nd'
 {
    "class" :"Distribut

In [49]:
# d) d. India losing, batting 2nd, Ashwin not playing
print (model.predict_proba({'result': 'lost', 'batting': '2nd', 'ashwin_playing': 'N'}))
# Gives the Result probability
print (model.predict_proba({'batting': '2nd', 'ashwin_playing': 'N'}))

[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.0,
            "Away" :1.0
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'N' '2nd' 'lost']
[{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "Home" :0.0,
            "Away" :1.0
        }
    ],
    "frozen" :false
}
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "lost" :0.8974358974358972,
            "won" :0.10256410256410274
        }
    ],
    "frozen" :false
}
 'N' '2nd'
 {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution"